In [1]:
FN = 'nea_extended.json'

from bulk_lambert.bulk_many import propagate_many

import json

from joblib import Parallel, delayed

from astropy.time import Time
from astropy import units as u

from poliastro.bodies import Sun
from poliastro.frames import Planes
from poliastro.twobody import Orbit

from orbital.utilities import true_anomaly_from_mean

import numpy as np

In [2]:
def orbit_from_mpc(body):

    nu = true_anomaly_from_mean(
        e = body['e'],
        M = float((body['M'] * u.deg).to(u.rad).value)
    ) * u.rad

    if not -np.pi * u.rad <= nu < np.pi * u.rad:
        nu = ((nu + np.pi * u.rad) % (2 * np.pi * u.rad) - np.pi * u.rad).to(
            nu.unit
        )

    return Orbit.from_classical(
        Sun,
        a = (body['a'] * u.AU).to(u.m),
        ecc = body['e'] * u.one,
        inc = (body['i'] * u.deg).to(u.rad),
        raan = (body['Node'] * u.deg).to(u.rad),
        argp = (body['Peri'] * u.deg).to(u.rad),
        nu = nu,
        epoch = Time(body["Epoch"], format = 'jd'),
        plane = Planes.EARTH_ECLIPTIC,
    )

def read_mpc(fn):

    with open(fn, 'r') as f:
        raw = json.load(f)
        
    # return [orbit_from_mpc(body) for body in raw]
    return Parallel(n_jobs=8)(delayed(orbit_from_mpc)(body) for body in raw)

mpc_orbits = read_mpc(FN)

In [3]:
t = Time('2021-01-01 00:00')

In [22]:
# jit warmup & test

_a = mpc_orbits[0].propagate(t)
ar, av = _a.r.to(u.km).value, _a.v.to(u.km / u.s).value
_br, _bv = propagate_many(mpc_orbits[:1], t)
br, bv = _br.to(u.km).value, _bv.to(u.km / u.s).value

np.all(np.isclose(ar, br)), np.all(np.isclose(av, bv))

(True, True)

In [26]:
def propagate_classic(orbits, epoch):
    return [orbit.propagate(epoch) for orbit in orbits]

%timeit propagate_classic(mpc_orbits, t)

35.7 s ± 736 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit propagate_many(mpc_orbits, t)

3.38 s ± 38.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
len(mpc_orbits)

20250

Propagating a single orbit with `propagate_cpu` to 20250 epochs requires 93.9 ms.

In [30]:
3380 / 92.9

36.38320775026911